Based on tensorflow starter code from https://www.kaggle.com/alexozerin/end-to-end-baseline-tf-estimator-lb-0-72

In [1]:
import os
import re
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
POSSIBLE_LABELS = 'yes no up down left right on off stop go silence unknown'.split()
id2name = {i: name for i, name in enumerate(POSSIBLE_LABELS)}
name2id = {name: i for i, name in id2name.items()}
print(name2id)

{'yes': 0, 'down': 3, 'silence': 10, 'left': 4, 'right': 5, 'stop': 8, 'no': 1, 'up': 2, 'unknown': 11, 'go': 9, 'on': 6, 'off': 7}


In [3]:
def load_data(data_dir):
    
    all_files = glob(os.path.join(data_dir, 'train/audio/*/*wav'))

    with open(os.path.join(data_dir, 'train/validation_list.txt'), 'r') as fin:
        validation_files = fin.readlines()
    
    # Just a simple regexp for paths with three groups:
    # prefix, label, user_id
    pattern = re.compile("(.+\/)?(\w+)\/([^_]+)_.+wav")
    valset = set()
    for entry in validation_files:
        r = re.match(pattern, entry)
        if r:
            valset.add(r.group(3))

    possible = set(POSSIBLE_LABELS)
    train, val = [], []
    for entry in all_files:
        r = re.match(pattern, entry)
        if r:
            label, uid = r.group(2), r.group(3)
            if label == '_background_noise_':
                label = 'silence'
            if label not in possible:
                label = 'unknown'

            label_id = name2id[label]

            sample = (label, label_id, uid, entry)
            if uid in valset:
                val.append(sample)
            else:
                train.append(sample)

    print('There are {} train and {} val samples'.format(len(train), len(val)))
    
    columns_list = ['label', 'label_id', 'user_id', 'wav_file']
    
    train_df = pd.DataFrame(train, columns = columns_list)
    valid_df = pd.DataFrame(val, columns = columns_list)
    
    return train_df, valid_df

In [4]:
train_df, valid_df = load_data('./data/')

There are 57929 train and 6798 val samples


In [5]:
train_df.head()

,label,label_id,user_id,wav_file
0,yes,0,8eb4a1bf,./data/train/audio/yes/8eb4a1bf_nohash_3.wav
1,yes,0,9b402bc2,./data/train/audio/yes/9b402bc2_nohash_0.wav
2,yes,0,20174140,./data/train/audio/yes/20174140_nohash_0.wav
3,yes,0,3b8406c0,./data/train/audio/yes/3b8406c0_nohash_1.wav
4,yes,0,9c59dd28,./data/train/audio/yes/9c59dd28_nohash_2.wav


In [6]:
train_df.label.value_counts()

unknown    36818
stop        2134
yes         2116
up          2115
go          2112
right       2111
on          2110
left        2106
no          2105
off         2101
down        2095
silence        6
Name: label, dtype: int64

In [7]:
# Separating silence data
silence_files = train_df[train_df.label == 'silence']
train_df      = train_df[train_df.label != 'silence']

In [8]:
from scipy.io import wavfile
def read_wav_file(fname):
    _, wav = wavfile.read(fname)
    wav = wav.astype(np.float32) / np.iinfo(np.int16).max
    return wav

In [9]:
# Combining the silence data
silence_data = np.concatenate([read_wav_file(x) for x in silence_files.wav_file.values])
len(silence_data)/16000 # in seconds

/home/sainath/anaconda3/lib/python3.5/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


399.3981875

In [10]:
from scipy.signal import stft

def calc_phase_amp(wav):
    specgram = stft(wav, 16000, nperseg = 400, noverlap = 240, nfft = 512, padded = False, boundary = None)
    phase = np.angle(specgram[2]) / np.pi
    amp = np.log1p(np.abs(specgram[2]))
    return np.stack([phase, amp], axis = 2)

def process_wav_file(fname):
    wav = read_wav_file(fname)
    
    L = 16000  # 1 sec
    
    if len(wav) > L:
        i = np.random.randint(0, len(wav) - L) # if length is more than 1s, randomly sample the file
        wav = wav[i:(i+L)]
    elif len(wav) < L: # if length is less than 1s, add silence on both ends to make it 1s
        rem_len = L - len(wav)
        i = np.random.randint(0, len(silence_data) - rem_len)
        silence_part = silence_data[i:(i+L)]
        j = np.random.randint(0, rem_len)
        silence_part_left  = silence_part[0:j]
        silence_part_right = silence_part[j:rem_len]
        wav = np.concatenate([silence_part_left, wav, silence_part_right])
    
    return calc_phase_amp(wav)

In [11]:
import random
import tensorflow as tf
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, Conv2D, MaxPooling2D, Activation, BatchNormalization, GlobalAveragePooling2D, GlobalMaxPool2D, concatenate, Dense, Dropout
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.utils import to_categorical

In [12]:
def train_generator(train_batch_size):
    while True:
        this_train = train_df.copy()
        
        # let's say we have about 400 silence files (since we have ~400s of silence data)
        ids = list(range(this_train.shape[0]))
        ids = ids + [None]*400
        
        shuffled_ids = random.sample(ids, len(ids))
        for start in range(0, len(shuffled_ids), train_batch_size):
            x_batch = []
            y_batch = []
            
            end = min(start + train_batch_size, len(shuffled_ids))
            
            i_train_batch = shuffled_ids[start:end]
            
            for i in i_train_batch:
                if i is None:
                    k = np.random.randint(0, len(silence_data) - 16000)
                    this_silence = silence_data[k:(k+16000)]
                    x_batch.append(calc_phase_amp(this_silence))
                    y_batch.append(name2id['silence'])
                else:
                    x_batch.append(process_wav_file(this_train.wav_file.values[i]))
                    y_batch.append(this_train.label_id.values[i])
            
            x_batch = np.array(x_batch)
            y_batch = to_categorical(y_batch, num_classes = len(POSSIBLE_LABELS))
            yield x_batch, y_batch

In [13]:
def valid_generator(val_batch_size):
    while True:
        ids = list(range(valid_df.shape[0]))
        for start in range(0, len(ids), val_batch_size):
            x_batch = []
            y_batch = []
            end = min(start + val_batch_size, len(ids))
            i_val_batch = ids[start:end]
            for i in i_val_batch:
                x_batch.append(process_wav_file(valid_df.wav_file.values[i]))
                y_batch.append(valid_df.label_id.values[i])
            x_batch = np.array(x_batch)
            y_batch = to_categorical(y_batch, num_classes = len(POSSIBLE_LABELS))
            yield x_batch, y_batch

In [14]:
x_in = Input(shape = (257,98,2))
x = BatchNormalization()(x_in)
for i in range(4):
    x = Conv2D(16*(2 ** i), (3,3))(x)
    x = Activation('elu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)
x = Conv2D(128, (1,1))(x)
x_branch_1 = GlobalAveragePooling2D()(x)
x_branch_2 = GlobalMaxPool2D()(x)
x = concatenate([x_branch_1, x_branch_2])
x = Dense(128, activation = 'relu')(x)
x = Dropout(0.5)(x)
x = Dense(len(POSSIBLE_LABELS), activation = 'softmax')(x)
model = Model(inputs = x_in, outputs = x)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 257, 98, 2)    0                                            
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 257, 98, 2)    8           input_1[0][0]                    
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 255, 96, 16)   304         batch_normalization_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 255, 96, 16)   0           conv2d_1[0][0]                   
___________________________________________________________________________________________

In [15]:
# from keras_tqdm import TQDMNotebookCallback
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [16]:
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=4,
                           verbose=1,
                           min_delta=0.01,
                           mode='min'),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=2,
                               verbose=1,
                               epsilon=0.01,
                               mode='min'),
             ModelCheckpoint(monitor='val_loss',
                             filepath='weights/02-weights.hdf5',
                             save_best_only=True,
                             save_weights_only=True,
                             mode='min')]

In [19]:
(train_df.shape[0] + 400) / 128

455.6484375

In [20]:
(valid_df.shape[0])/128

53.109375

In [21]:
history = model.fit_generator(generator=train_generator(128),
                              steps_per_epoch=456,
                              epochs=25,
                              verbose=2,
                              callbacks=callbacks,
                              validation_data=valid_generator(128),
                              validation_steps=54)

Epoch 1/25
273s - loss: 1.4869 - categorical_accuracy: 0.6320 - val_loss: 4.0851 - val_categorical_accuracy: 0.1403
Epoch 2/25
258s - loss: 0.8002 - categorical_accuracy: 0.7457 - val_loss: 0.5796 - val_categorical_accuracy: 0.8071
Epoch 3/25
257s - loss: 0.4991 - categorical_accuracy: 0.8413 - val_loss: 0.5486 - val_categorical_accuracy: 0.8441
Epoch 4/25
259s - loss: 0.3825 - categorical_accuracy: 0.8817 - val_loss: 0.4293 - val_categorical_accuracy: 0.8754
Epoch 5/25
255s - loss: 0.3194 - categorical_accuracy: 0.9016 - val_loss: 0.2933 - val_categorical_accuracy: 0.9134
Epoch 6/25
263s - loss: 0.2776 - categorical_accuracy: 0.9147 - val_loss: 0.5409 - val_categorical_accuracy: 0.8704
Epoch 7/25
249s - loss: 0.2487 - categorical_accuracy: 0.9251 - val_loss: 0.4256 - val_categorical_accuracy: 0.9011
Epoch 8/25

Epoch 00007: reducing learning rate to 0.00010000000474974513.
257s - loss: 0.2307 - categorical_accuracy: 0.9304 - val_loss: 0.4185 - val_categorical_accuracy: 0.9026
Epoch 9/

In [22]:
model.load_weights('./weights/02-weights.hdf5')

In [23]:
test_paths = glob(os.path.join('./data/', 'test/audio/*wav'))

In [24]:
def test_generator(test_batch_size):
    while True:
        for start in range(0, len(test_paths), test_batch_size):
            x_batch = []
            end = min(start + test_batch_size, len(test_paths))
            this_paths = test_paths[start:end]
            for x in this_paths:
                x_batch.append(process_wav_file(x))
            x_batch = np.array(x_batch)
            yield x_batch

In [25]:
predictions = model.predict_generator(test_generator(256), int(np.ceil(len(test_paths)/256)))

In [26]:
classes = np.argmax(predictions, axis=1)

In [27]:
# last batch will contain padding, so remove duplicates
submission = dict()
for i in range(len(test_paths)):
    fname, label = os.path.basename(test_paths[i]), id2name[classes[i]]
    submission[fname] = label

In [28]:
with open('submission2.csv', 'w') as fout:
    fout.write('fname,label\n')
    for fname, label in submission.items():
        fout.write('{},{}\n'.format(fname, label))

In [29]:
pd.DataFrame([(key, submission[key]) for key in submission], columns=['file', 'label']).label.value_counts()

unknown    98727
silence     7288
off         6410
no          5912
left        5552
yes         5436
stop        5041
on          5006
up          4814
down        4799
right       4793
go          4760
Name: label, dtype: int64